In [3]:
from gmft.pdf_bindings import PyPDFium2Document
from gmft import CroppedTable, TableDetector

detector = TableDetector()

def ingest_pdf(pdf_path) -> list[CroppedTable]:
    doc = PyPDFium2Document(pdf_path)

    tables = []
    for page in doc:
        tables += detector.extract(page)
    return tables, doc

In [4]:
# tables, doc = ingest_pdf("example_report.pdf")
tables, doc = ingest_pdf("/home/snexus/Downloads/2407.03291v1.pdf")

In [5]:
type(tables[0])

gmft.table_detection.CroppedTable

In [6]:
len(doc)

26

In [7]:
type(doc)

gmft.pdf_bindings.bindings_pdfium.PyPDFium2Document

In [8]:
page = doc.get_page(24)
type(page)


gmft.pdf_bindings.bindings_pdfium.PyPDFium2Page

In [9]:
page.page_number

24

In [10]:
page_text = page.page.get_textpage()

In [11]:
print(page_text.get_text_range())

VCHAR:Variance-Driven Complex Human Activity Recognition framework with Generative Representation • 0 :25
[17] Harish Haresamudram, David V Anderson, and Thomas Plötz. 2019. On the role of features in human activity recognition. In Proceedings
of the 2019 ACM International Symposium on Wearable Computers. 78–88.
[18] Taeho Hur, Jaehun Bang, Thien Huynh-The, Jongwon Lee, Jee-In Kim, and Sungyoung Lee. 2018. Iss2Image: A novel signal-encoding
technique for CNN-based human activity recognition. Sensors 18, 11 (2018), 3910.
[19] Sozo Inoue, Paula Lago, Tahera Hossain, Tittaya Mairittha, and Nattaya Mairittha. 2019. Integrating activity recognition and nursing
care records: The system, deployment, and a verification study. Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous
Technologies 3, 3 (2019), 1–24.
[20] Chihiro Ito, Xin Cao, Masaki Shuzo, and Eisaku Maeda. 2018. Application of CNN for human activity recognition with FFT spectrogram
of acceleration and gyro sensors.

/home/snexus/projects/llm-search/.venv/lib64/python3.11/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [12]:
# p1 = doc.page
# help(p1.get_positions_and_text())

In [13]:
doc

In [14]:
# for table in tables:
#     table.visualize(figsize=None)

In [15]:
from gmft import AutoTableFormatter, TATRTableFormatter

formatter = TATRTableFormatter()

In [16]:
help(formatter.extract)

Help on method extract in module gmft.table_function:

extract(table: gmft.table_detection.CroppedTable, dpi=144, padding='auto') -> gmft.table_function.FormattedTable method of gmft.table_function.TATRTableFormatter instance
    Extract the data from the table.



In [21]:
table_index = 2
table = tables[table_index]

In [24]:
table

TypeError: CroppedTable.from_image_only() missing 1 required positional argument: 'img'

In [20]:

ft = formatter.extract(tables[table_index], dpi=200)
ft.visualize()

ValueError: The identified boxes have significant overlap: 216.11% of area is overlapping (Max is 20.00%)

In [66]:
ft = formatter.extract(tables[table_index])

In [67]:
# ft.to_dict()

In [68]:
ft.df().info()

ValueError: The identified boxes have significant overlap: 215.94% of area is overlapping (Max is 20.00%)

In [27]:
df = ft.df()

In [34]:
import re
df.columns =  [re.sub(r"\W+","", col)+str(i) for i, col in enumerate(df.columns) ]

In [42]:
df

,0,m1,m2
0,Shop fits for new and existing stores,1.3,1.3
1,Production equipment and tooling,9.3,10.1
2,Computer equipment and software,2.1,2.9
3,Site,1.9,3.4
4,Total capital additions,14.6,17.7


In [36]:
ft.df().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5 non-null      object
 1   m1      5 non-null      object
 2   m2      5 non-null      object
dtypes: object(3)
memory usage: 252.0+ bytes


In [46]:
df.to_records()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>m1</th>\n      <th>m2</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Shop fits for new and existing stores</td>\n      <td>1.3</td>\n      <td>1.3</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Production equipment and tooling</td>\n      <td>9.3</td>\n      <td>10.1</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Computer equipment and software</td>\n      <td>2.1</td>\n      <td>2.9</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Site</td>\n      <td>1.9</td>\n      <td>3.4</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Total capital additions</td>\n      <td>14.6</td>\n      <td>17.7</td>\n    </tr>\n  </tbody>\n</table>'

In [47]:
def func(row):
    xml = ['<item>']
    for field in row.index:
        xml.append('  <field name="{0}">{1}</field>'.format(field, row[field]))
    xml.append('</item>')
    return '\n'.join(xml)

In [49]:
df.apply(func, axis=1)

0    <item>\n  <field name="0">Shop fits for new an...
1    <item>\n  <field name="0">Production equipment...
2    <item>\n  <field name="0">Computer equipment a...
3    <item>\n  <field name="0">Site</field>\n  <fie...
4    <item>\n  <field name="0">Total capital additi...
dtype: object

In [48]:
print('\n'.join(df.apply(func, axis=1)))

<item>
  <field name="0">Shop fits for new and existing stores</field>
  <field name="m1">1.3</field>
  <field name="m2">1.3</field>
</item>
<item>
  <field name="0">Production equipment and tooling</field>
  <field name="m1">9.3</field>
  <field name="m2">10.1</field>
</item>
<item>
  <field name="0">Computer equipment and software</field>
  <field name="m1">2.1</field>
  <field name="m2">2.9</field>
</item>
<item>
  <field name="0">Site</field>
  <field name="m1">1.9</field>
  <field name="m2">3.4</field>
</item>
<item>
  <field name="0">Total capital additions</field>
  <field name="m1">14.6</field>
  <field name="m2">17.7</field>
</item>


In [22]:
ft.df().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0                                                 5 non-null      object
 1   Number of stores at 29 May 2022               6 non-null      object
 2   Opened                                        6 non-null      object
 3   Closed                                        6 non-null      object
 4   Number of stores at 28 May 2023               6 non-null      object
 5   Number of single staff stores at 28 May 2023  6 non-null      object
 6   Number of single staff stores at 29 May 2022  6 non-null      object
dtypes: object(7)
memory usage: 468.0+ bytes
